# Wrangle and Analyze WeRateDogs Data

"The dataset that you will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage."

### Table of Contents

<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
    <ul>
    <li><a href="#gathering">Gathering Data</a></li>
    <li><a href="#assessing">Assessing Data</a></li>
        <ul>
          <li><a href="#issues">Identified Issues</a></li>
        </ul>
    <li><a href="#cleaning">Cleaning Data</a></li>
    </ul>
<li><a href="#sav">Store, Analyze and Visualize</a></li>
<li><a href="#conclusion">Conclusion</a></li>
</ul>

<a id='intro'></a>
## Introduction

dd

#### Import

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tweepy
import json
import matplotlib as plt
%matplotlib inline

# https://stackoverflow.com/questions/11707586/how-do-i-expand-the-output-display-to-see-more-columns
# To see all rows in datasets in order to help visual assessment
# pd.set_option('display.max_rows', 5000)

<a id='wrangling'></a>
## Data Wrangling

In this section of the report, we will load in the data, check it for cleanliness, and then trim and clean the datasets for analysis. 

<a id='gathering'></a>
### Gathering Data

#### Enhanced WeRateDogs Dataset 

In [4]:
# Read in the dataset
enhanced_df = pd.read_csv('twitter-archive-enhanced.csv')

#### Dog Breed Predictions

In [5]:
# Create folder if it doesn't already exist
folder_name = 'tweet_image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [6]:
# Load dataset and check response
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response.status_code

200

In [7]:
# Add data to folder
with open(os.path.join(folder_name, url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)
    
os.listdir(folder_name)

['image-predictions.tsv']

In [8]:
# Read in tsv as csv
predictions_df = pd.read_csv(folder_name + '/image-predictions.tsv', sep='\t')

#### Accessing Twitter API

In [9]:
# # API Keys, Secrets, and Tokens
# consumer_key = ''
# consumer_secret = ''
# access_token = ''
# access_secret = ''

In [10]:
# # Redirect to Twitter and get access token
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

# # API instance
# # guidance for this was found here: https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id
# api = tweepy.API(auth_handler=auth, 
#                  wait_on_rate_limit=True, 
#                  wait_on_rate_limit_notify=True)


In [11]:
# # Twitter API

# # Split into working tweets list and list of tweets with errors
# tweets = []
# errors = []

# tweet_ids = list(enhanced_df['tweet_id'])

# with open('tweet_json.txt', 'w') as file:
#     for tweet_id in tweet_ids:
#         try:
#             # Get extended tweet information via the id
#             extended_tweet = api.get_status(tweet_id, tweet_mode='extended')
#             json.dump(extended_tweet._json, file)
#             file.write('\n')
            
#             # Add to working tweets list
#             tweets.append(tweet_id)
#             print(tweet_id)
        
#         # Support used to better understand TweepError: https://www.programcreek.com/python/example/13279/tweepy.TweepError
#         except tweepy.TweepError as e:
            
#             # Add to list of tweets with errors
#             errors.append(tweet_id)
#             print(tweet_id, e)

Note: do not include your Twitter API keys, secrets, and tokens in your project submission.

In [12]:
# Read in JSON 
tweet_df = pd.read_json('tweet_json.txt', lines = True, encoding = 'utf-8')

<a id='assessing'></a>
### Assessing Data

In this section, we will visually and programmatically assess the 3 datasets to determine whether or not they hold any quality or tidiness issues.

#### Enhanced twitter dataframe assessment

In [13]:
# Overview of all the data
enhanced_df

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [92]:
# See amount of data within each column, how many rows exist, and the datatypes
enhanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [ ]:
# Check if texts are robust
enhanced_df.text[400]

In [ ]:
# See how many images are missing
enhanced_df.expanded_urls.isnull().sum()

In [ ]:
# See how many images are duplicated
enhanced_df.expanded_urls.duplicated().sum()

In [ ]:
# Find missing data
enhanced_df.isnull().sum()

In [ ]:
# Find number of retweets
enhanced_df.retweeted_status_id.notna().sum()

In [ ]:
# Look at dog names to ensure they are real names
enhanced_df.name.value_counts()

In [ ]:
# How many doggos are there
enhanced_df.doggo.value_counts()

In [ ]:
# How many floofers are there
enhanced_df.floofer.value_counts()

In [ ]:
# How many puppers are there
enhanced_df.pupper.value_counts()

In [ ]:
# How many puppos are there
enhanced_df.puppo.value_counts()

#### Predicted dog type assessment

In [ ]:
# Overview of all the data
predictions_df

In [ ]:
# See amount of data within each column, how many rows exist, and the datatypes
predictions_df.info()

In [ ]:
predictions_df.p1.str.islower().sum()

In [ ]:
predictions_df.jpg_url.duplicated().sum()

In [ ]:
predictions_df.tweet_id.duplicated().sum()

In [ ]:
predictions_df.p1.value_counts()

#### Twitter API data assessment

In [ ]:
# Overview of all the data
tweet_df

In [ ]:
# See amount of data within each column, how many rows exist, and the datatypes
tweet_df.info()

In [ ]:
# Check for missing values
tweet_df.isnull().sum()

In [ ]:
tweet_df.id.duplicated().sum()

<a id='issues'></a>
### Identified Issues

#### Quality Issues

##### `enhanced_df` table:
- Retweet information is not needed, so items that are not NaN in `retweet_status_id` can be removed
- Irrelevant columns since we only want to look at original ratings (`in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`)
- Missing photo URLs for tweets (`expanded_url`) 
- Erroneous datatype (`timestamp` and `retweeted_status_timestamp` are strings) 
- Erroneous dog names ('a', 'an', 'the', 'his', 'such', 'just', 'very', 'one', 'not', 'mad', 'this', 'all','my', 'light', 'by', 'old', 'space', 'officially' and None are not real names)
- Duplicated rows after melting

##### `predictions_df` table:
- About half of `p1`, `p2`, and `p3` are not capitalized
- Duplicated images/rows
- Images predictions of things that are not dogs

##### `tweet_df` table:
- Missing data for entire column (`contributors`,`coordinates`,`geo`)
- Missing data for almost entire column (`in_reply_to_screen_name`,`in_reply_to_status_id`,`in_reply_to_status_id_str`,`in_reply_to_status_id_str`,`in_reply_to_user_id`,`in_reply_to_user_id_str`,`place`,`quoted_status`, `quoted_status_id`,`quoted_status_id_str`, `quoted_status_permalink`, `retweeted_status`)

#### Tidiness Issues
- Single variable `dog_stage` split up into four columns in `enhanced_df` table
- Column name not clear(`id` should be `tweet_id` to match the other tables) in `tweet_df` table
- All tables can be combined into one on `tweet_id`

<a id='cleaning'></a>
### Cleaning Data

In this section, we will target the quality and tidiness issues were identified in the previous section, and we will define these issues in more depth, code the solution, and test to ensure proper functionality. 

Before we begin, we should create copies of our dataframes in order to not alter the originals.

In [14]:
# Create copies
enhanced_df_clean = enhanced_df.copy()
predictions_df_clean = predictions_df.copy()
tweet_df_clean = tweet_df.copy()

#### Retweet information is not needed, so items that are not NaN in `retweet_status_id` can be removed
##### Define
Change the datatype of column to string since it is easier to query strings than deal with floats. Then, create a new dataframe with all of the 'nan' values, which indicate that they are not retweets. 

##### Code

In [107]:
# Change `retweeted_status_id` into a string datatype
enhanced_df_clean.retweeted_status_id = enhanced_df_clean.retweeted_status_id.astype('str')

In [108]:
# Update dataframe with just the nan values
enhanced_df_cleaner = enhanced_df_clean[(enhanced_df_clean.retweeted_status_id == 'nan')]

##### Test

In [110]:
# Check to see if there are any retweets left
enhanced_df_cleanerer.retweeted_status_id.value_counts()

nan    2175
Name: retweeted_status_id, dtype: int64

#### Irrelevant columns since we only want to look at original ratings (`in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`)
##### Define
Now that the retweeted dog ratings are removed, we can drop all retweet related columns.

##### Code

In [111]:
# Drop all irrelevant columns
enhanced_df_clean.drop(columns=[
    'in_reply_to_status_id', 
    'in_reply_to_user_id', 
    'retweeted_status_id', 
    'retweeted_status_user_id', 
    'retweeted_status_timestamp'], axis=1, inplace=True)

##### Test

In [112]:
# Check the columns remaining
list(enhanced_df_clean)

['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

#### Single variable `dog_stage` split up into four columns in `enhanced_df` table
##### Define
To put all the dog stages into one column, first we will split the `enhanced_df_clean` table into a df with the dog stages and into a df without the dog stages. For the dataframe without dog stages, we will drop all the dog stage columns. For the dataframe with dog stages, we will melt the columns into one with the results. Then we will concatenate the two cleaned dataframes to create the new `enhanced_df_clean`.

##### Code

In [113]:
# Create dataframe without any dog stages
without_stage = enhanced_df_clean[(enhanced_df_clean.doggo == 'None') 
                                  & (enhanced_df_clean.floofer == 'None') 
                                  & (enhanced_df_clean.pupper == 'None') 
                                  & (enhanced_df_clean.puppo == 'None')]

##### Test

In [114]:
# Check to see that none of the rows have any dog stages
without_stage[(without_stage.doggo != 'None') 
                & (without_stage.floofer != 'None') 
                & (without_stage.pupper != 'None') 
                & (without_stage.puppo != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [115]:
# Check to see the total rows for dogs without a stage
without_stage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1976 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              1976 non-null int64
timestamp             1976 non-null object
source                1976 non-null object
text                  1976 non-null object
expanded_urls         1923 non-null object
rating_numerator      1976 non-null int64
rating_denominator    1976 non-null int64
name                  1976 non-null object
doggo                 1976 non-null object
floofer               1976 non-null object
pupper                1976 non-null object
puppo                 1976 non-null object
dtypes: int64(3), object(9)
memory usage: 200.7+ KB


##### Code

In [116]:
# Drop the dog stage columns since they now have been checked to ensure nothing is within them
without_stage.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

/Users/gabriela/Desktop/Code/myenv/lib/python3.7/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


##### Test

In [117]:
# Check to see if the columns were correctly dropped
without_stage.head(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin


In [118]:
enhanced_df_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


##### Code

In [119]:
# Create a dataframe for all dogs with a stage
with_stage = enhanced_df_clean[(enhanced_df_clean.doggo != 'None') 
                                  | (enhanced_df_clean.floofer != 'None') 
                                  | (enhanced_df_clean.pupper != 'None') 
                                  | (enhanced_df_clean.puppo != 'None')]

##### Test

In [121]:
# Check samples to ensure that each dog has a stage
with_stage.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1797,677269281705472000,2015-12-16 23:29:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is the happiest pupper I've ever seen. 10...,https://twitter.com/dog_rates/status/677269281...,10,10,the,None,None,pupper,None
860,763167063695355904,2016-08-10 00:16:21 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Eve. She's a raging alcoho...,https://twitter.com/dog_rates/status/673295268...,8,10,Eve,None,None,pupper,None
440,819924195358416896,2017-01-13 15:08:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a doggo who has messed up. He was...,https://twitter.com/dog_rates/status/819924195...,11,10,None,doggo,None,None,None
1051,742534281772302336,2016-06-14 01:49:03 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","For anyone who's wondering, this is what happe...",https://vine.co/v/iLTZmtE1FTB,11,10,None,doggo,None,None,None
581,800443802682937345,2016-11-20 21:00:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sampson. He's about to ...,https://twitter.com/dog_rates/status/776113305...,11,10,Sampson,None,None,pupper,None


In [122]:
# Melt the dog stage columns of doggo, floofer, pupper, and puppo into a single column
with_stage = pd.melt(with_stage, id_vars=['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name'],
                           var_name='dog_stage', value_name='dog_type')

##### Test

In [123]:
# Check sample to see the results
with_stage.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,dog_type
1501,673707060090052608,2015-12-07 03:34:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Raymond. He's absolutely terrified of ...,https://twitter.com/dog_rates/status/673707060...,10,10,Raymond,puppo,None
987,717790033953034240,2016-04-06 19:04:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a badass mystery pupper. You weren't aw...,https://twitter.com/dog_rates/status/717790033...,10,10,None,pupper,pupper
1189,829878982036299777,2017-02-10 02:25:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Loki. He smiles like El...,https://twitter.com/dog_rates/status/826958653...,12,10,Loki,puppo,None
1232,803773340896923648,2016-11-30 01:31:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Diogi. He fell in the pool as soon as ...,https://twitter.com/dog_rates/status/803773340...,12,10,Diogi,puppo,puppo
922,755955933503782912,2016-07-21 02:41:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a frustrated pupper attempting to escap...,https://twitter.com/dog_rates/status/755955933...,12,10,None,pupper,pupper


In [124]:
# Check to see if there's repetition with the name since the indices are different
with_stage[(with_stage.name == 'Rinna')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,dog_type
327,679777920601223168,2015-12-23 21:37:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rinna. She's melting. 10/10 get inside...,https://twitter.com/dog_rates/status/679777920...,10,10,Rinna,doggo,None
707,679777920601223168,2015-12-23 21:37:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rinna. She's melting. 10/10 get inside...,https://twitter.com/dog_rates/status/679777920...,10,10,Rinna,floofer,None
1087,679777920601223168,2015-12-23 21:37:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rinna. She's melting. 10/10 get inside...,https://twitter.com/dog_rates/status/679777920...,10,10,Rinna,pupper,pupper
1467,679777920601223168,2015-12-23 21:37:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rinna. She's melting. 10/10 get inside...,https://twitter.com/dog_rates/status/679777920...,10,10,Rinna,puppo,None


##### Code

In [125]:
# Drop the rows with None since those are just duplicates, and then drop the dog stage
with_stage = with_stage[with_stage.dog_type != "None"]
with_stage = with_stage.drop('dog_stage', axis=1)

##### Test

In [126]:
# Check sample to ensure it worked out
with_stage.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type
1160,867421006826221569,2017-05-24 16:44:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Shikha. She just watched you drop a sk...,https://twitter.com/dog_rates/status/867421006...,12,10,Shikha,puppo
1123,673656262056419329,2015-12-07 00:12:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Albert AKA King Banana Peel. He's a ki...,https://twitter.com/dog_rates/status/673656262...,10,10,Albert,pupper
41,834089966724603904,2017-02-21 17:18:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",DOGGO ON THE LOOSE I REPEAT DOGGO ON THE LOOSE...,https://twitter.com/stevekopack/status/8340866...,10,10,None,doggo
957,741743634094141440,2016-06-11 21:27:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Aqua. She's a sandy pupper. Not sure how ...,https://twitter.com/dog_rates/status/741743634...,11,10,Aqua,pupper
1028,693629975228977152,2016-01-31 03:00:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This pupper is afraid of its own feet. 12/10 w...,https://twitter.com/dog_rates/status/693629975...,12,10,None,pupper


##### Code

In [127]:
# Concatenate the with and without stage dataframes back into a joined dataframe
enhanced_df_clean = pd.concat([with_stage, without_stage], join='outer', sort=False)

##### Test

In [135]:
# Check to see if it concatenated correctly
enhanced_df_clean.sample(10)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type
2200,668655139528511488,2015-11-23 04:59:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Winifred. He is a Papyrus Hydrang...,https://twitter.com/dog_rates/status/668655139...,11,10,Winifred,NaN
450,818646164899774465,2017-01-10 02:30:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Seamus. He's very bad a...,"https://vine.co/v/5QWd3LZqXxd,https://vine.co/...",11,10,Seamus,NaN
2082,670815497391357952,2015-11-29 04:04:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sage. He likes to burn shit. 10/10 htt...,https://twitter.com/dog_rates/status/670815497...,10,10,Sage,NaN
1681,681981167097122816,2015-12-29 23:32:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jimothy. He's a Trinidad Poliwhirl. Fa...,https://twitter.com/dog_rates/status/681981167...,12,10,Jimothy,NaN
1504,691820333922455552,2016-01-26 03:09:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Brockly. He's an uber driver. Falls as...,https://twitter.com/dog_rates/status/691820333...,8,10,Brockly,NaN
1632,684460069371654144,2016-01-05 19:42:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jeph. He's a Western Sagittarius Dookm...,https://twitter.com/dog_rates/status/684460069...,10,10,Jeph,NaN
671,789960241177853952,2016-10-22 22:42:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Buddy. His father was a...,https://twitter.com/dog_rates/status/762464539...,12,10,Buddy,NaN
150,863062471531167744,2017-05-12 16:05:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Quinn. She's quite the goofball. ...,"https://www.gofundme.com/helpquinny,https://tw...",13,10,Quinn,NaN
562,802600418706604034,2016-11-26 19:50:26 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Bailey. She has mastered the head tilt...,https://vine.co/v/5FwUWjYaW0Y,11,10,Bailey,NaN
1132,672898206762672129,2015-12-04 22:00:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cheryl AKA Queen Pupper of the Skies. ...,https://twitter.com/dog_rates/status/672898206...,11,10,Cheryl,pupper


In [136]:
# Check to see if all the dog stages are still present
enhanced_df_clean.dog_type.value_counts()

pupper     257
doggo       97
puppo       30
floofer     10
Name: dog_type, dtype: int64

In [137]:
# The following 4 are to ensure the cleaned dataframes dog stages match the original dataframe
enhanced_df.pupper.value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

In [138]:
enhanced_df.doggo.value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

In [139]:
enhanced_df.puppo.value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [140]:
enhanced_df.floofer.value_counts()

None       2346
floofer      10
Name: floofer, dtype: int64

In [141]:
enhanced_df_clean.dog_type.isna().sum()

1976

In [142]:
###### Clean
enhanced_df_clean.tweet_id.duplicated().sum()

14

In [143]:
##### Clean
enhanced_df_clean[enhanced_df_clean.tweet_id.duplicated()]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type
408,854010172552949760,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...","At first I thought this was a shy doggo, but i...",https://twitter.com/dog_rates/status/854010172...,11,10,None,floofer
837,817777686764523521,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",https://twitter.com/dog_rates/status/817777686...,13,10,Dido,pupper
849,808106460588765185,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,https://twitter.com/dog_rates/status/808106460...,12,10,None,pupper
854,802265048156610565,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",https://twitter.com/dog_rates/status/802265048...,11,10,None,pupper
857,801115127852503040,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,pupper
880,785639753186217984,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,pupper
885,781308096455073793,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",https://vine.co/v/5rgu2Law2ut,12,10,None,pupper
896,775898661951791106,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Like father (doggo), like son (...",https://twitter.com/dog_rates/status/733109485...,12,10,None,pupper
905,770093767776997377,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is just downright precious...,https://twitter.com/dog_rates/status/741067306...,12,10,just,pupper
915,759793422261743616,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Meet Maggie &amp; Lila. Maggie is the doggo, L...",https://twitter.com/dog_rates/status/759793422...,12,10,Maggie,pupper


#### Column name not clear(`id` should be `tweet_id` to match the other tables) in `tweet_df` table
##### Define

##### Code

In [ ]:
tweet_df_clean.rename(columns={'id':'tweet_id'}, inplace=True)

##### Test

In [ ]:
list(tweet_df_clean)

#### All tables can be combined into one on `tweet_id`
##### Define

##### Code

In [ ]:
# Merge the three dataframes to create a master dataframe
twitter_archive_master = enhanced_df_clean.merge(predictions_df_clean, how='inner', on='tweet_id').merge(tweet_df_clean, how='inner', on='tweet_id')

##### Test

In [ ]:
list(twitter_archive_master)

### Assess - Twitter Archive Master

In [ ]:
twitter_master_copy.dog_type.isna().sum()

In [ ]:
# Create a copy to continue cleaning the data
twitter_master_copy = twitter_archive_master.copy

# hh

Duplicated rows after melting

In [ ]:
# enhanced_df_clean.tweet_id.duplicated().index
duplicate_dogs = twitter_archive_master[twitter_archive_master.tweet_id.duplicated()]
duplicate_dogs

In [ ]:
twitter_archive_master.tweet_id = twitter_archive_master.tweet_id.astype('str')

In [ ]:
# twitter_archive_master[twitter_archive_master['tweet_id'] == '775898661951791106']

In [ ]:
# Visually assess the duplicates to see if they can be classified and kept
dog_list = list(duplicate_dogs.index)

for x in dog_list:
    print('>', twitter_archive_master.tweet_id[x], '-', twitter_archive_master.text[x], '\n')

In [ ]:
twitter_archive_master.drop

#### 1. Missing photo URLs for tweets (`expanded_urls`) in the `enhanced_df` table
##### Define

Since we only want original ratings that have images, the tweets that are missing photo URLs will be dropped. 

##### Code

In [ ]:
enhanced_df_clean.expanded_urls.dropna(inplace=True)

#### Test

In [ ]:
enhanced_df_clean.expanded_urls.isnull().sum()

#### 2. Erroneous datatype (`timestamp` and `retweeted_status_timestamp` are strings) in the `enhanced_df` table
##### Define
The datatype of `timestamp` and `retweeted_status_timestamp` in the `enhanced_df` table are strings but it should be datetime.

#### Code

In [ ]:
# Datatime information found here: https://stackoverflow.com/questions/17134716/convert-dataframe-column-type-from-string-to-datetime
enhanced_df_clean.timestamp = enhanced_df_clean.timestamp.astype('datetime64')
enhanced_df_clean.retweeted_status_timestamp = enhanced_df_clean.retweeted_status_timestamp.astype('datetime64')

##### Test

In [ ]:
enhanced_df_clean.info()

#### 3. Erroneous dog names ('a', 'an', 'the', 'such', 'his', 'quite' and None are not real names) in the `enhanced_df` table
##### Define
Since 'a', 'an', 'the', 'such', 'his', 'quite' and None are not names, we should preferably replace these with NaN as to not lead to confusion, and to identify that the name is not provided.

##### Code

In [144]:
enhanced_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2370 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id              2370 non-null int64
timestamp             2370 non-null object
source                2370 non-null object
text                  2370 non-null object
expanded_urls         2311 non-null object
rating_numerator      2370 non-null int64
rating_denominator    2370 non-null int64
name                  2370 non-null object
dog_type              394 non-null object
dtypes: int64(3), object(6)
memory usage: 185.2+ KB


In [148]:
# https://stackoverflow.com/questions/17097236/replace-invalid-values-with-none-in-pandas-dataframe
enhanced_df_clean.name.replace(['a','an','the','such','his','None','quite'], np.nan, inplace=True)

##### Test

In [149]:
enhanced_df_clean.name.value_counts()
####### Not working

Charlie         12
Oliver          11
Cooper          11
Lucy            11
Lola            10
Penny           10
Tucker          10
Bo               9
Winston          9
Sadie            8
Buddy            7
Bailey           7
Daisy            7
Toby             7
Milo             6
Jack             6
Scout            6
Dave             6
Bella            6
just             6
Oscar            6
Leo              6
Koda             6
Rusty            6
Jax              6
Stanley          6
Oakley           5
Gus              5
Louis            5
Finn             5
                ..
Erik             1
DayZ             1
Birf             1
Tino             1
Sparky           1
Mimosa           1
Jonah            1
Brutus           1
Gilbert          1
Mac              1
Kota             1
Banditt          1
Lorelei          1
Bobby            1
Tedrick          1
Carbon           1
Jackie           1
Bert             1
Sora             1
Brandonald       1
Karl             1
Rumpole     

#### 5. About half of `p1`, `p2`, and `p3` are not capitalized in the `predictions_df` table
##### Define

Capitalize all data in `p1`, `p2`, and `p3` within the `predictions_df_clean` table.

##### Code

In [150]:
predictions_df_clean.p1 = predictions_df_clean.p1.str.capitalize()
predictions_df_clean.p2 = predictions_df_clean.p2.str.capitalize()
predictions_df_clean.p3 = predictions_df_clean.p3.str.capitalize()

##### Test

In [151]:
predictions_df_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
511,676191832485810177,https://pbs.twimg.com/media/CWJQ4UmWoAIJ29t.jpg,2,Chihuahua,0.376741,True,Italian_greyhound,0.173114,True,Muzzle,0.071485,False
255,670755717859713024,https://pbs.twimg.com/media/CU8AwZ_UsAA-Lbu.jpg,1,Keeshond,0.994065,True,Norwegian_elkhound,0.001827,True,Cairn,0.001821,True
1788,829861396166877184,https://pbs.twimg.com/media/C4RCiIHWYAAwgJM.jpg,1,Border_terrier,0.394486,True,Staffordshire_bullterrier,0.376574,True,American_staffordshire_terrier,0.031292,True
1117,725786712245440512,https://pbs.twimg.com/media/ChKDKmIWIAIJP_e.jpg,1,Chow,0.335761,True,Samoyed,0.167173,True,Kuvasz,0.145715,True
1168,735991953473572864,https://pbs.twimg.com/media/CjbExRKUoAAs089.jpg,2,Cocker_spaniel,0.961643,True,Toy_poodle,0.011547,True,Soft-coated_wheaten_terrier,0.004903,True


#### 5. Duplicated images/rows in the `predictions_df` table
##### Define
Identify the data that are duplicated in the `predictions_df` table and keep the first data of the duplicates set.

##### Code

In [ ]:
predictions_df_clean.duplicated(keep='first')

##### Test

In [ ]:
predictions_df_clean.duplicated().sum()

#### 6. Missing data for entire column (`contributors`,`coordinates`,`geo`) in `tweet_df` table
##### Define
Drop `contributors`, `coordinates`, and `geo` columns in `tweet_df` table

##### Code

In [ ]:
# Dropping columns correctly: https://stackoverflow.com/questions/21457917/pandas-dataframe-dropped-column-appearing-again
tweet_df_clean.drop(columns=['contributors', 'coordinates', 'geo'], axis=1, inplace=True)

##### Test

In [ ]:
# Confirm columns are gone
list(tweet_df_clean)

#### 7. Missing data for almost entire column (`in_reply_to_screen_name`,`in_reply_to_status_id`,`in_reply_to_status_id_str`,`in_reply_to_status_id_str`,`in_reply_to_user_id`,`in_reply_to_user_id_str`,`place`,`quoted_status`, `quoted_status_id`,`quoted_status_id_str`, `quoted_status_permalink`, `retweeted_status`) in `tweet_df` table
##### Define
Since these columns do not play a major role and are not relevant for the analyses and visualizations that we will conduct, they will be dropped as well. 

##### Code

In [ ]:
tweet_df_clean.drop(columns=['in_reply_to_screen_name','in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_status_id_str','in_reply_to_user_id','in_reply_to_user_id_str','place','quoted_status', 'quoted_status_id','quoted_status_id_str', 'quoted_status_permalink', 'retweeted_status'], axis=1, inplace=True)

##### Test

In [ ]:
# Confirm columns are gone
list(tweet_df_clean)

<a id='sav'></a>
## Store, Analyze & Visualize

Store the clean DataFrame(s) in a CSV file with the main one named twitter_archive_master.csv. If additional files exist because multiple tables are required for tidiness, name these files appropriately. Additionally, you may store the cleaned data in a SQLite database (which is to be submitted as well if you do).

Analyze and visualize your wrangled data in your wrangle_act.ipynb Jupyter Notebook. At least three (3) insights and one (1) visualization must be produced.

<a id='conclusion'></a>
## Conclusion

>Create a 300-600 word written report called wrangle_report.pdf or wrangle_report.html that briefly describes your wrangling efforts. This is to be framed as an internal document.

>Create a 250-word-minimum written report called act_report.pdf or act_report.html that communicates the insights and displays the visualization(s) produced from your wrangled data. This is to be framed as an external document, like a blog post or magazine article, for example.

Throughout this notebook, we can see... 

A thorough explanation of .. can be found in `wrangle_report.pdf` and more detail into .. can be found in `act_report.pdf`. 